## Compartment model, mean response D-optimal

In [1]:
using PyPlot
using LinearAlgebra
using NPZ

#objective function h to minimize
function infmat(x)
    n = Int((length(x)+1)/2)
    a = x[1:n]
    b = x[(n+1):(2*n-1)]
    b = [b; 1-sum(b)]
    p = length(theta)
    mat = zeros(p,p)
    for i in 1:n
        c1 = a[i]*theta[3]*exp(-theta[1]*a[i])
        c2 = -a[i]*theta[3]*exp(-theta[2]*a[i]) 
        c3 = exp(-theta[2]*a[i])-exp(-theta[1]*a[i])
        f = [c1, c2, c3]
        mat = mat + b[i] .* f*f'
    end
    return mat
end

# projection function onto the design space
function proj(x, design)
    n = Int((length(x)+1)/2)
    a = x[1:n]
    b = x[(n+1):(2*n-1)]
    b = [b; 1-sum(b)]
    
    a = max.(a,design[1])
    a = min.(a,design[2])
    
    u = sort(b, rev=true)
    j = n
    while true
        if u[j] + 1/j*(1-cumsum(u)[j]) > 0
            break
        end
        j -= 1
    end
    lambda = 1/j*(1-cumsum(u)[j])
    b = max.(b .+ lambda,0)
    b = b[1:(n-1)]
    
    return [a;b]
end

proj (generic function with 1 method)

In [2]:
function diter(k,nptc,niter,design)
    tau = 0.9:(-0.5/niter):0.4
    v = zeros(nptc, 2k-1)
    x = (design[2]-design[1])*rand(nptc*k) .+ design[1]
    x = reshape(x,nptc,k)
    w = rand(nptc*(k-1))
    w = reshape(w,nptc,k-1)
    ptc = hcat(x,w)
    for j in 1:nptc
        ptc[j,:] = proj(ptc[j,:],design)
    end
    pbest = ptc
    pbesth = zeros(nptc)
    for i in 1:nptc
        pbesth[i] = det(infmat(pbest[i,:]))
    end
    gbesth = findmax(pbesth)[1]
    gbest = pbest[findmax(pbesth)[2],:]

    #iterate
    for i in 1:niter # iteration number
        for j in 1:nptc # particle number
            v[j,:] = tau[i] .*v[j,:] + 2 .*rand(2k-1) .*(pbest[j,:]-ptc[j,:]) + 2 .*rand(2k-1) .*(gbest-ptc[j,:])
            ptc[j,:] = ptc[j,:] + v[j,:]
            ptc[j,:] = proj(ptc[j,:],design)
            fit = det(infmat(ptc[j,:]))
            if fit > gbesth
                gbest = pbest[j,:] = ptc[j,:]
                gbesth = pbesth[j] = fit
            elseif fit > pbesth[j]
                pbest[j,:] = ptc[j,:]
                pbesth[j] = fit
            end
        end
    end
    supp = gbest[1:k]
    prob = gbest[(k+1):2k-1]
    prob = [prob; 1-sum(prob)]
    return [supp, prob, gbesth ]
end

diter (generic function with 1 method)

In [3]:
theta = [0.05884, 4.298, 21.8]
#k = 3
#nptc = 100
#niter = 100
#design = [0,30]

result = diter(3, 100, 100, [0,30])


3-element Array{Any,1}:
     [0.228772, 1.38858, 18.4168]  
     [0.333333, 0.333331, 0.333336]
 1617.589538599137                 

## number of support points vs CPU time

In [4]:
result = zeros(100)
for nk = 1:100
    t1 = time()
    diter(nk, 100, 100, [0,30])
    t2 = time()
    result[nk] = t2-t1
end

In [5]:
result

100-element Array{Float64,1}:
 0.1371619701385498 
 0.14742779731750488
 0.16645193099975586
 0.19301795959472656
 0.21195721626281738
 0.23247289657592773
 0.2619650363922119 
 0.28763389587402344
 0.316295862197876  
 0.3304741382598877 
 0.3616788387298584 
 0.39202213287353516
 0.40885496139526367
 ⋮                  
 2.6282260417938232 
 2.643889904022217  
 2.606149196624756  
 2.6962509155273438 
 2.1192450523376465 
 2.000659942626953  
 2.0430550575256348 
 2.0232410430908203 
 2.1000449657440186 
 2.102008104324341  
 2.1281769275665283 
 2.1983580589294434 

In [6]:
npzwrite("scpu.npy",result)